In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = (
 '/home/kathia/Desktop/objectiv/objectiv-analytics/bach/.secrets/objectiv-production--bigquery-read-only.json'
)

In [3]:
from sqlalchemy import create_engine
from bach import DataFrame, display_sql_as_markdown, SeriesDict, SeriesList
engine = create_engine('bigquery://objectiv-production/')

df = DataFrame.from_table(
    engine=engine,
    table_name='events',
    index=[],
    bq_project_id='objectiv-production',
    bq_dataset='snowplow',
)
df_base = df.copy()
df.head()

,app_id,platform,etl_tstamp,collector_tstamp,dvce_created_tstamp,event,event_id,txn_id,name_tracker,v_tracker,...,event_vendor,event_name,event_format,event_version,event_fingerprint,true_tstamp,load_tstamp,contexts_io_objectiv_taxonomy_1_0_0,contexts_com_snowplowanalytics_snowplow_ua_parser_context_1_0_0,contexts_nl_basjes_yauaa_context_1_0_2
0,None,web,2022-05-12 09:08:36.564,2022-05-12 09:08:36.403,NaT,struct,43ee6b3d-8365-4840-8643-ca952de1bf15,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,b934921f3147d00cb53f976db0549cf8,NaT,2022-05-12 09:08:37.428611,"[{'_type': 'MediaLoadEvent', '_types': '[""Abst...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."
1,None,web,2022-05-12 09:08:36.564,2022-05-12 09:08:36.404,NaT,struct,66148ea5-1112-4131-a06b-bdaf401d4ba3,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,af41314e04b9c2a29e88a9cac4945eab,NaT,2022-05-12 09:08:37.428665,"[{'_type': 'ApplicationLoadedEvent', '_types':...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."
2,None,web,2022-05-12 09:08:37.513,2022-05-12 09:08:37.451,NaT,struct,f3398916-a71c-4c8f-b8b8-02a3fc3d39c6,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,9f90a314e2bf77fb85f826a06942adb6,NaT,2022-05-12 09:08:38.367620,"[{'_type': 'MediaLoadEvent', '_types': '[""Abst...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."
3,None,web,2022-05-12 09:13:41.380,2022-05-12 09:13:39.998,NaT,struct,e2cd1b08-dd6c-49f6-9be7-f350e43908fd,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,c10aabc334d690995e7bfcd4b6cf9430,NaT,2022-05-12 09:13:42.471425,"[{'_type': 'VisibleEvent', '_types': '[""Abstra...","[{'device_family': 'Other', 'os_family': 'Wind...","[{'device_class': 'Desktop', 'agent_build': No..."
4,None,web,2022-05-12 09:13:41.380,2022-05-12 09:13:39.999,NaT,struct,7779c038-1264-4e6b-bd7d-1b5347279305,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,2b235c5e3bb863960ef44f5ad751d283,NaT,2022-05-12 09:13:42.471455,"[{'_type': 'VisibleEvent', '_types': '[""Abstra...","[{'device_family': 'Other', 'os_family': 'Wind...","[{'device_class': 'Desktop', 'agent_build': No..."


In [13]:
df.hits.insta

ValueError: Invalid key: hit_number. Available keys: ['appInfo', 'contentGroup', 'contentInfo', 'customDimensions', 'customMetrics', 'customVariables', 'dataSource', 'eCommerceAction', 'eventInfo', 'exceptionInfo', 'experiment', 'hitNumber', 'hour', 'isEntrance', 'isExit', 'isInteraction', 'isSecure', 'item', 'latencyTracking', 'minute', 'page', 'product', 'promotion', 'promotionActionInfo', 'publisher', 'publisher_infos', 'referer', 'refund', 'social', 'sourcePropertyInfo', 'time', 'transaction', 'type']

In [20]:
df.hits.elements[0].elements['hitNumber'].view_sql()

'with `from_table___00d9fdfb5298012a6b53d183fc4d8dd2` as (SELECT * FROM `objectiv-production`.`95166736`.`ga_sessions_20201116`)\nselect `hits`[OFFSET(0)].`hitNumber` as `hits` \nfrom `from_table___00d9fdfb5298012a6b53d183fc4d8dd2` \n \n \n \n \n \n'

In [4]:
sub_df = df[['event_id', 'contexts_io_objectiv_taxonomy_1_0_0']]
#x['y'] = x['contexts_io_objectiv_taxonomy_1_0_0'].elements[0]
sub_df['nested_event_id'] = sub_df['contexts_io_objectiv_taxonomy_1_0_0'].elements[0].elements['event_id']
sub_df['cmp'] = sub_df['event_id'] == sub_df['nested_event_id']
sub_df.head()

,event_id,contexts_io_objectiv_taxonomy_1_0_0,nested_event_id,cmp
0,e27313af-f1e0-4d1f-99cd-fe205406361a,"[{'_type': 'ApplicationLoadedEvent', '_types':...",e27313af-f1e0-4d1f-99cd-fe205406361a,True
1,aa3c32e3-8852-478d-828a-0310dfe995a9,"[{'_type': 'MediaLoadEvent', '_types': '[""Abst...",aa3c32e3-8852-478d-828a-0310dfe995a9,True
2,c8f572e8-5cf1-46ea-8996-7acc55cecfa3,"[{'_type': 'PressEvent', '_types': '[""Abstract...",c8f572e8-5cf1-46ea-8996-7acc55cecfa3,True
3,8075cb5f-499c-464f-9590-851c44c6797a,"[{'_type': 'VisibleEvent', '_types': '[""Abstra...",8075cb5f-499c-464f-9590-851c44c6797a,True
4,4e826236-8360-4d50-8a37-2909bfe7f448,"[{'_type': 'PressEvent', '_types': '[""Abstract...",4e826236-8360-4d50-8a37-2909bfe7f448,True


In [5]:
df.head()

,app_id,platform,etl_tstamp,collector_tstamp,dvce_created_tstamp,event,event_id,txn_id,name_tracker,v_tracker,...,event_vendor,event_name,event_format,event_version,event_fingerprint,true_tstamp,load_tstamp,contexts_io_objectiv_taxonomy_1_0_0,contexts_com_snowplowanalytics_snowplow_ua_parser_context_1_0_0,contexts_nl_basjes_yauaa_context_1_0_2
0,None,web,2022-03-21 07:12:55.252,2022-03-21 07:12:55.062,NaT,struct,50eb4d73-373e-430d-b5b8-032aa81ecd4e,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,3c77947273c3d324caf07f94498eb09a,NaT,2022-03-21 07:12:56.093855,"[{'_type': 'ApplicationLoadedEvent', '_types':...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."
1,None,web,2022-03-21 07:12:56.280,2022-03-21 07:12:56.222,NaT,struct,a6492efb-71dd-4d26-ad96-e6b525fc787b,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,80921ec55e451bcac17e9572116a36da,NaT,2022-03-21 07:12:56.948567,"[{'_type': 'MediaLoadEvent', '_types': '[""Abst...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."
2,None,web,2022-03-21 07:19:28.326,2022-03-21 07:19:28.158,NaT,struct,26b841b5-955e-4230-bb5f-898d5a998189,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,d2e738ea6253e57a784014fd599c0ecd,NaT,2022-03-21 07:19:29.113671,"[{'_type': 'ApplicationLoadedEvent', '_types':...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."
3,None,web,2022-03-21 07:19:29.387,2022-03-21 07:19:29.322,NaT,struct,0bc0965d-f4b4-4e4b-b2ea-2f79b51d1910,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,17a6f7ba87c4e188a7f8d7082a6f46be,NaT,2022-03-21 07:19:30.062392,"[{'_type': 'MediaLoadEvent', '_types': '[""Abst...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."
4,None,web,2022-03-21 12:05:48.432,2022-03-21 12:05:48.238,NaT,struct,5021a0c9-615b-43a4-9012-34f95d6079fa,None,None,objectiv-tracker-0.0.5,...,com.google.analytics,event,jsonschema,1-0-0,2afc5ec3e162fd7b0fc86bc6e9de2b33,NaT,2022-03-21 12:05:49.332370,"[{'_type': 'PressEvent', '_types': '[""Abstract...","[{'device_family': 'Mac', 'os_family': 'Mac OS...","[{'device_class': 'Desktop', 'agent_build': No..."


In [6]:
df = DataFrame.from_table(
    engine=engine,
    table_name='ga_sessions_20201116',
    index=[],
    bq_project_id='objectiv-production',
    bq_dataset='95166736',
)
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,None,1,1605544231,1605544231,20201116,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '(not set)', 'campaign': 'axa...","{'browser': 'Edge', 'browserVersion': '86.0.62...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 5, 'value': ''}, {'index': 36, 'val...","[{'hitNumber': 1, 'time': 0, 'hour': 16, 'minu...",6884936558251780385,None,1603024210.1605544225,Text Message,Not Socially Engaged
1,None,4,1605569172,1605569172,20201116,"{'visits': None, 'hits': 1, 'pageviews': None,...","{'referralPath': '(not set)', 'campaign': 'pi-...","{'browser': 'Chrome', 'browserVersion': '85.0....","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 5, 'value': ''}, {'index': 36, 'val...","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",4718636731542343940,None,1098643227.1605439748,Email,Not Socially Engaged
2,None,8,1605541733,1605541733,20201116,"{'visits': None, 'hits': 1, 'pageviews': None,...","{'referralPath': '(not set)', 'campaign': 'pi-...","{'browser': 'Safari', 'browserVersion': '14.0'...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 5, 'value': ''}, {'index': 36, 'val...","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...",7099535024397750252,None,1652989309.1605111788,Email,Not Socially Engaged
3,None,2,1605558303,1605558303,20201116,"{'visits': None, 'hits': 1, 'pageviews': None,...","{'referralPath': '(not set)', 'campaign': 'pi-...","{'browser': 'Safari', 'browserVersion': '13.1....","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 5, 'value': ''}, {'index': 36, 'val...","[{'hitNumber': 1, 'time': 0, 'hour': 20, 'minu...",320082984800794964,None,74525127.1605548372,Email,Not Socially Engaged
4,None,8,1605560043,1605560043,20201116,"{'visits': None, 'hits': 1, 'pageviews': None,...","{'referralPath': '(not set)', 'campaign': 'pi-...","{'browser': 'Chrome', 'browserVersion': '86.0....","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 36, 'value': 'customer'}, {'index':...","[{'hitNumber': 1, 'time': 0, 'hour': 20, 'minu...",795030982466946064,None,185107575.1600078864,Email,Not Socially Engaged
